<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/My_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Do not try to install with conda in Colab
!pip install -q huggingface_hub
!pip install -q transformers datasets
!pip install wandb

import json
import numpy as np
import os
import sys
import torch
import wandb

from datasets              import DatasetDict
from google.colab          import auth, files, userdata
from huggingface_hub       import create_repo, HfApi, login, upload_file
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from transformers          import AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, Trainer, TrainingArguments



In [ ]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
print(f"currentdir: {os.getcwd()}")
print()

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")


currentdir: /content

device: cpu


In [ ]:
### Google Colab authentication

# Required for accessing Colab Secrets
auth.authenticate_user()


In [ ]:
### Hugging Face authentications

login(token=userdata.get('HF_TOKEN'))

# Verify
!huggingface-cli whoami


claudelepere


In [ ]:
### Create the skill_classification repo on the Hugging Face Hub

"""
Save anything related to the FINAL artifacts of the project:
- Model weights (e.g., pytorch_model.bin or tf_model_h5)
- Tokenizer and vocabulary files (e.g., tokenizer.json, vocal.txt)
- Config files (e.g., config.json, preprocessor.json)
- Datasets for training and inference (if small or public)
- Model cards describing the model (e.g., README.md)
"""

name            = "claudelepere/skill_classification"
repo_id_model   = name
repo_id_dataset = name

api = HfApi()

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
    )
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
    )
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")


Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [ ]:
### Weights & Biases (W&B, wandb) authentication

"""
Track the training process and experiment management:
- Metrics: Log training and validation loss, accuracy, F1 scores, etc.
- Hyperparameters: save configurations for the experiments (learning rate, batch size)
- Training artifacts:
  - Intermediate model checkpoints
  - Training logs or charts
- Data versions: save different datasets splits
"""

wandb.login(key=userdata.get('WANDB_API_KEY'))
wandb.init(project="skill_classification", entity="claudelepere")

# My fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a **linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels)**, indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [ ]:
### Upload and unzip the dataset zip file

uploaded_files     = files.upload()
uploaded_file_name = list(uploaded_files.keys())[0]
print(f"uploaded_file_name: {uploaded_file_name} {len(uploaded_files)}")

!unzip {uploaded_file_name}

unzipped_file_dir_name = os.path.splitext(uploaded_file_name)[0]
print(f"unzipped_file_dir_name: {unzipped_file_dir_name}")


In [ ]:
### Create the dataset

dataset = DatasetDict.load_from_disk(unzipped_file_dir_name)

print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's test the first example of the training split:

In [ ]:
example = dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

In [ ]:
### Create the label list and the id2label and label2id mappings.

"""
labels
    if dataset 7_1000_125_125  , 48 labels
    if dataset 7_128_18_54     , 42 labels
    if dataset 8910_1087_68_204, 206 labels
"""

labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")


The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
### Upload the label list as a JSON file on the HF repo

labels_path = "labels.json"
with open(labels_path, 'w') as f:
    json.dump(labels, f)
print(f"labels saved to {labels_path}")

repo_labels_path  = "labels.json"
upload_file(
    path_or_fileobj = labels_path,
    path_in_repo    = repo_labels_path,
    repo_id         = repo_id_labels,
    repo_type       = "dataset"
    )
print(f"labels uploaded to https://huggingface.co/datasets/{repo_id_labels}/tree/main/{repo_labels_path}")


## Preprocess data

As models like BERT don't expect text as direct input, but rather **`input_ids`**, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a **matrix of shape (batch_size, num_labels)**. Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' **BCEWithLogitsLoss** (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

### Preprocess (examples, not example, because batched=True => examples is a batch)

In [ ]:
### Tokenize 'text'

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def preprocess_data(examples, indices):
  text = examples['text']    # Batch of texts

  encoding = tokenizer(
      text,                             # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = 512,
      return_tensors = 'pt'             # Return PyTorch tensors
      )

  # Create an empty label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Add labels to the encoding
  encoding['labels'] = labels_matrix
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding


In [ ]:
### Create the encoded dataset

encoded_dataset = dataset.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset['train'].column_names,
    with_indices   = True
    )

In [ ]:
example = encoded_dataset['validation'][0]
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

In [ ]:
tokenizer.decode(example['input_ids'])

In [ ]:
example['labels']

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [ ]:
encoded_dataset.set_format("torch")    # Ensures correctness and compatibility with PyTorch pipelines

## Define model

Here we define a **model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top**. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [**BCEWithLogitsLoss**](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
### Define the model

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
    )

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size  = 8
metric_name = "f1"

### TrainingArguments

In [ ]:
args = TrainingArguments(
    output_dir                  = "/content/results/output",
    overwrite_output_dir        = True,
    logging_dir                 = "/content/results/logs",
    logging_steps               = 50,
    save_steps                  = 100,
    save_total_limit            = 2,
    eval_strategy               = "epoch",
    save_strategy               = "epoch",
    learning_rate               = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    num_train_epochs            = 5,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    #push_to_hub                 = True,
    run_name                   = "BERT-multilabel-lr2e5-epochs5-datasetHF_128_18_54"
    )

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
### Metrics

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics


In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result


Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()
encoded_dataset['train']['input_ids'][0]

In [ ]:
print(f"inputids:       {type(encoded_dataset['train']['input_ids'][0])}      {encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask: {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels:         {type(encoded_dataset['train'][0]['labels'])}         {encoded_dataset['train'][0]['labels'].shape}")

In [ ]:
### Execute the forward pass

outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

outputs


Let's start training!

In [ ]:
### Create the trainer

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )

In [ ]:
### Train

train_ouput = trainer.train()
print(f"train_ouput.global_step: {type(train_ouput.global_step)} {train_ouput.global_step}")        # Total training steps
print(f"train_ouput.training_loss: {type(train_ouput.training_loss)} {train_ouput.training_loss}")  # Final training loss
print(f"train_ouput.metrics: {type(train_ouput.metrics)} {train_ouput.metrics}")                    # Training metrics


## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
### Evaluate

eval_results = trainer.evaluate()

In [ ]:
api.upload_file(
    path_or_fileobj = "eval_results.json",
    path_in_repo    = "eval_results.json",
    repo_id         = repo_id,
    repo_type       = "model",

In [ ]:
### Save:
### - the trained model and the tokenizer (saves the model weights, the tokenizer, the model configuration file ("config.json"))
### - the train and evaluation metrics

model_path         = "skills_model"
train_metrics_path = "train_metrics.json"
eval_metrics_path  = "eval_metrics.json"


In [ ]:
trainer.save_model(model_path)

with open(train_metrics_path, 'w') as f:
  json.dump(trainer.state.log_history,f)

with open(eval_metrics_path, 'w') as f:
  json.dump(eval_results, f)


In [ ]:
### Upload the model and the tokenizer

model     = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.push_to_hub(repo_id_model)
tokenizer.push_to_hub(repo_id_model)

In [ ]:
raise Exception("STOP")

## Inference

Let's test the model on a new sentence:

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

## device

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

Download labels.json

In [ ]:
import json
from huggingface_hub import hf_hub_download

In [ ]:
repo_id_labels = "claudelepere/skills_labels"
repo_filename  = "labels.json"

In [ ]:
file_path = hf_hub_download(repo_id                = repo_id_labels,
                            filename               = repo_filename,
                            repo_type              = "dataset",
                            local_dir              = "/content",
                            local_dir_use_symlinks = False
                           )
with open(file_path, 'r') as f:
    labels = json.load(f)
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

Upload skills.csv

In [ ]:
from google.colab import files
uploaded = files.upload()
!ls -la

## Filtered skills (those in labels)

In [ ]:
import pandas as pd

In [ ]:
skill_df          = pd.read_csv("skills.csv")
skill_df['Id']    = skill_df['Id'].astype(str)
skill_df['Value'] = skill_df['Value'].astype(str)
filtered_skill_df = skill_df[skill_df['Id'].isin(labels)]
print(f"filtered_skill_df: {type(filtered_skill_df)} {filtered_skill_df.shape}\n{filtered_skill_df}")

### Model and Tokenizer from the model repo

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [ ]:
repo_id_model = "claudelepere/skills_model"

In [ ]:
# Load the trained model and tokenizer
model     = AutoModelForSequenceClassification.from_pretrained(repo_id_model)
tokenizer = AutoTokenizer.from_pretrained(repo_id_model)

### New texts to classify

text = "I'm happy, I can finally train a model for multi-label classification."
preds: none

In [ ]:
#### in train or validation or test
# id = 23553
text = "Cream Consulting - Business Analyst Insurance, Property&Casualty (P&C) Cream Consulting Today we are looking for Business Analyst to extend our team specialized in Business Process Improvement. You will join a multi-lingual team focusing on P&C business. In that position, you will define and clarify the role and responsibilities. You will be responsible for leading business process reviews where you will identify and implement innovative solutions. You will define and document the new processes and systems to meet our client's business objectives. What the job is all about? Implement new solutions for P&C insurance companies Write and validate the “as is” Plan the analysis of organization's strategic business needs Understand End Users needs Develop process modeling and design Follow up projects and monitor development Organize and conduct testing What Cream is looking for? At least 5 years of experience in Business Analysis in insurance business Analysis know-how Fluent editorial Good knowledge of Property & Casualty insurance business to interface with all stakeholders Functional knowledge of insurance products Good communication skills Fluent English, good knowledge in French and/or Dutch is a plus"
# "356,168,149,795,802,137,139,138,353"; SkillTypeId 7: true: 149, 168, 356 preds: 142, 170

### NL
id = 323697
text = "Talencia Consulting - Senior Full Stack Developer (Java & Angular) Full Stack, DevOps, AKS, OAuth, API Talencia Consulting Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"
"142,189,190,754,208,794,676,811,812,139,138"; SkillTypeId: true: 142; preds: 142, 170

In [ ]:
#### EN
# id = 323611
text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
# "171,170,794,800,798,797,138,139,352"; SkillTypeId 7: true: 170, 171; preds: 142, 170

id = 323526
text = "Vivid Resourcing - Chief Technology Officer CTO, reliability, business goals Vivid Resourcing We're partnered with a leading sustainability-oriented company near Brussels, aiming to combat high pollution rates worldwide. They are currently working on a unique application that rewards workers for reducing their carbon footprint, whilst also maintaining and even improving profits. Together we are seeking a visionary Chief Technology Officer (CTO) who aligns with their mission and ambitions. The ideal candidate will possess a strong hands-on technical background, proven management experience, and strong business acumen. This role requires a strategic thinker who can drive technological direction and support the company's growth objectives of transitioning from a scale-up to an established business entity, so any past experience leading teams in this manner would go a long way. Key responsibilities Develop and execute the company's technological vision and strategy Lead and mentor a team of engineers and technologists Oversee all technical aspects of the company, ensuring alignment with business goals Drive innovation in regenerative sustainable technologies and carbon measurement systems Collaborate with cross-functional teams to integrate technology solutions Ensure the reliability, security, and scalability of technological infrastructures Foster a culture of continuous improvement and technical excellence Qualifications Experience leading a team within a small to medium sized company Strong technical background in software development/data analytics/system architecture Bachelor's or Master's degree in either an IT or Business related field Experience in the agriculture or environmental sectors is a plus Proven management skills with the ability to lead, communicate and inspire a diverse team Excellent business acumen and strategic thinking Strong problem-solving skills and the ability to make informed decisions in a fast-paced environment Offer Taking charge of a genuinely impactful product, using your direction for the good of the environment Complete responsibility over a technical team, with management responsibilities Up to 110,000 EUR gross for experienced applicants, which can then be increased further Full benefits package including mobility costs Flexible hybrid work Inclusive work environment If this role interests you, attach a CV and apply today!"
"175,169,139,352"; SkillTypeId 7: true: 169, 175 preds: 142, 170

### FR
id = 323517
text = "AG Insurance - Technical Architect REST API, DevOps AG Insurance Un lieu de travail où vous pouvez prendre des initiatives. Où tout le monde vous encourage à montrer ce dont vous êtes capable. Où on vous encourage à vous développer. Et où vous construisez l'avenir avec vos collègues. Vous avez de l'audace et le client est au centre de vos préoccupations ? Alors, à très bientôt chez AG ! Notre compagnie compte environ 700 spécialistes IT. Autant de collègues géniaux qui contribuent chaque jour à la (r)évolution technologique chez AG. Le département Information System (IS) est responsable du développement des applications supportant le business assurance, ainsi que des applications transversales dans les domaines suivants : In & Outbounds, Referentials & Financials, sans oublier des centres d'expertise technologique pour le développement des Front-Ends, pour la mise en place d'APIs et s... Et si c'était votre prochain job ? Nous recherchons un(e) .NET Solution Architect pour renforcer ce département. Vous proposez des solutions techniques adaptées aux besoins et aux contraintes des projets et conformes à la vision IT d'AG. En innovant et en améliorant ce qui existe déjà, vous créez également une valeur ajoutée pour AG. Pendant la phase architecturale, vous choisissez les solutions en concertation avec les autres architectes techniques et fonctionnels, et avec toutes les autres parties concernées (autres équipes, partenaires externes, collègues d'Infrastructure & Operations, etc.). Vous concevez l'architecture technique des applications sur la base des exigences fonctionnelles et non fonctionnelles. Nous comptons également sur vous pour le support technique, du développement à la production de l'application. Vous assurez le suivi de l'implémentation de vos solutions, en coopération avec les équipes de développement. Vous vous assurez ensuite que la solution mise en œuvre est conforme à l'architecture proposée. Si nécessaire, vous apportez des corrections. Vous accompagnez et coachez vos collègues pour les aider à monter en compétence dans le domaine. Ce poste vous intéresse ? Qu'attendez-vous pour postuler ? Nos recruteurs examineront avec vous quelle division de notre département IT et quelle équipe vous conviendront le mieux. Vous vous reconnaissez dans ce profil ? Les nouvelles technologies ? Vous en suivez de près l'évolution et les tendances en architecture IT. Vous avez une expérience solide dans les domaines suivants : Technical Architecture & Design Patterns REST API, .NET Framework & .NET 6 and up Source control tooling like Azure DevOps & GitHub Microsoft Azure Cloud SQL Server Vous êtes autonome et prenez les choses en main ? Bien sûr ! Mais vous savez remonter les informations ou demander du support si nécessaire. Team-player dans l'âme, vous partagez naturellement votre savoir-faire avec vos collègues. La qualité du travail, dans ses moindres détails, c'est votre cheval de bataille. Vous le documentez avec précision, tant pour le support technique de votre équipe que pour les utilisateurs d'autres teams. Communiquer en français et/ou en néerlandais, à l'oral et à l'écrit ? Un jeu d'enfant pour vous ! Votre anglais est à la hauteur ? Encore mieux ! Un diplôme supérieur en informatique et/ou plus de 2 ans d'expérience dans une fonction similaire ? Foncez ! Vous n'allez pas rater ça ? Un super job chez le leader sur le marché de l'assurance. Et comme nous cherchons à encore mieux servir nos clients, nous comptons sur vous pour nous y aider. Un environnement de travail moderne dans tous les sens du terme : physique, digital et organisationnel. En d'autres termes, des heures de travail flexibles, la possibilité de télétravailler jusqu'à 3 jours par semaine et le matériel IT adéquat pour travailler de la maison. Une équipe enthousiaste et dynamique, notée 10/10 pour l'ambiance et la convivialité. La possibilité de vous perfectionner en continu, grâce à un large éventail de formations. Idéal pour apprendre toutes les compétences qui vous aideront à faire évoluer votre carrière. Une véritable carrière. En effet, travailler chez AG, c'est bien plus qu'un job. Envie d'explorer de nouveaux horizons après quelque temps ? Nous vous guidons et vous encourageons à exploiter tous vos talents à fond. Des bonnes vibrations, grâce à un vaste programme de bien-être bourré d'activités sportives et d'ateliers inspirants. De quoi vous sentir (encore) plus épanoui(e) dans votre travail et votre vie. Un lieu où tout le monde se sent bienvenu et a des chances égales de s'épanouir. Où votre avenir n'est pas déterminé par votre origine, votre âge, votre genre, votre orientation sexuelle ou votre handicap, mais par votre talent et vos compétences. Et comme l'argent, ça compte aussi : un package salarial attrayant. Vous pouvez même composer vous-même une partie de votre package, car personne ne sait mieux que vous ce dont vous avez besoin. Lancez-vous ! Emballé(e) ? > Postulez aujourd'hui encore ! Nous nous ferons un plaisir de vous aider si vous avez besoin de soutien avant ou pendant le processus de sélection."
"146,636,676,668,670,300,812,138,137"; SkillTypeId 7: true: 146; preds: 142, 170, 689

### Classify

In [ ]:
inputs    = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
threshold = 0.19

In [ ]:
# Perform the forward pass for inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

In [ ]:
# Convert logits to probabilities
probs = torch.sigmoid(logits)

In [ ]:
# Convert probabilities to predictions
preds = (probs > threshold).int()

In [ ]:
for label, Value, prob, pred in zip(filtered_skill_df['Id'], filtered_skill_df['Value'], probs.squeeze(), preds.squeeze()):
    print(f"label: {label} prob: {prob.item():.4f} pred: {int(pred.item())} {Value}")
print()
for label, Value, prob, pred in zip(filtered_skill_df['Id'], filtered_skill_df['Value'], probs.squeeze(), preds.squeeze()):
    if (pred == 1):
        print(f"label: {label} prob: {prob.item():.4f} pred: {int(pred.item())} {Value}")

### END #####################################################################

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
#text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
encoding = tokenizer(text, return_tensors="pt")

In [ ]:
# Define the device based on availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Move the model to the device
model.to(device)
# Move encoding to the device of the model
encoding = {k: v.to(device) for k,v in encoding.items()}

In [ ]:
# Perform inference
with torch.no_grad():    # no gradients needed for inference. Forward pass
    outputs = model(**encoding)

In [ ]:
# Get logits from the model's output
logits = outputs.logits

In [ ]:
# Apply softmax/sigmoid based on the type of classification
if model.config.num_labels == 1:
    probs = torch.sigmoid(logits.squeeze())
else:
    #probs = torch.softmax(logits, dim=1).squeeze()
    probs = torch.sigmoid(logits)

In [ ]:
# To get predictions
threshold = 0.5
#predictions = torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
#predictions = torch.argmax(probs, dim=-1) if model.config.num_labels > 1 else torch.where(probs >= threshold, torch.ones_like(probs), torch.zeros_like(probs))
predictions = (probs > threshold).float()
print("Predictions:", predictions)
print()

In [ ]:
# Turn predicted id's into actual label names
print("Probabilites:", probs)

[id2label[idx] for idx, label in enumerate(predictions['labels']) if label == 1.0]

predicted_labels = [id2label[idx.item()] for idx in predictions]
print(predicted_labels)

In [ ]:
for label, prob, pred in zip(labels, probs.squeeze(), predictions.squeeze()):
  print(f"Label: {label}: Probability: {prob.item():.4f} {int(pred.item())}")